# 6.1. Разнообразие

Как известно, для формирования случайного леса необходимо разнообразие каждого из решающих деревьев в его составе. Добиться такого разнообразия можно как при помощи варьирования параметров деревьев, так и при помощи варьирования обучающих выборок. Это обычно делается при помощи комбинирования методов случайных подпространств и беггинга. Метод случайных подпространств состоит в случайном выборе подмножества признаков. Бэггинг - это техника отбора подмножества объектов из исходной выборки, которая состоит в последовательном случайном выборе объектов выборки **с возвращением**.

Для ясности приведём следующий пример:

Пусть  наша выборка - это черный мешок с пронумерованными шарами. Каждый шар символизирует некоторый объект нашей выборки с соответствующим номером. Процедура бэггинга предлагает нам последовательно хорошо перемешав шары в мешке вытаскивать их один за другим не глядя, записывать их номера на лист бумаги, а затем возвращать их назад в мешок, повторяя эту операцию столько раз, сколько объектов содержится в нашей выборке. Затем мы вынем из мешка все те шары, номер которых хотя бы раз возник в нашем списке. Понятно, что скорее всего среди этих номеров будут повторяющиеся, то есть какие-то шары мы вытянем несколько раз, а какие-то - вообще ни разу. Именно поэтому наша подвыборка и получится случайной. Статистически обоснована оценка того, какой процент шаров из исходной выборки в среднем попадёт в итоговую подвыборку. Эта оценка приблизительно равна $62\%$.

Ваша задача - написать реализацию трёх функций и объединить их в класс `sample`, возвращающий по выборке некоторую случайную подвыборку, пригодную для обучения  одного из деревьев в ансамбле случайного леса.

**Замечание:** обратите внимание, что объекты в итоговой подвыборке не должны дублироваться. Мы предлагаем Вам ознакомиться с функциями [np.random.choice](https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html) и [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html), они могут оказаться полезны при выполнении этого задания. 

Подробнее об этом методе можно прочитать в нашей [лекции](https://colab.research.google.com/drive/1LrqEyfmOKJQdvgxZ56qPcJ_YjJFnP_Ka#scrollTo=kYjii_stqfJo) 


In [1]:
import numpy as np
np.random.seed(42)

class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)
  
    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X, random_state=42):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи бэггинга индексов.
        Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
        """
        return np.unique(np.random.choice(range(X.shape[0]), size=X.shape[0], replace=True))
  
    @staticmethod
    def random_subspace(X, n_subspace, random_state=42):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала массив индексов выбранных при помощи метода случайных подпространств признаков
        Количество этих признаков передается при помощи аргумента n_subspace
        Пользуйтесь только инструментами, реализованными в numpy.random, выставляя везде, где это необходимо, random_state=42
        """
        return np.random.choice(range(X.shape[1]), size=n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        """
        Заполните тело этой функции таким образом, чтобы она возвращала подвыборку x_sampled, y_sampled 
        по значениям индексов признаков(idx_subspace) и объектов(idx_obj) , которые должны в неё попасть
        """
        return X[idx_obj][:, idx_subspace], y[idx_obj]

# Пример формата входных и выходных данных

# 6.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле. 

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [113]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.3, random_state=random_state)

In [114]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier


N_ESTIMATORS = 1
MAX_DEPTH = 11
SUBSPACE_DIM = 1

class sample(object):
    def __init__(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)
  
    def __call__(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X):
        return np.unique(np.random.choice(range(X.shape[0]), size=X.shape[0], replace=True))
  
    @staticmethod
    def random_subspace(X, n_subspace):
        return np.random.choice(range(X.shape[1]), size=n_subspace, replace=False)

    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        return X[idx_obj][:, idx_subspace], y[idx_obj]


class random_forest(object):
    def __init__(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim
        self.random_state = random_state
        self._estimators_params = []
        self._estimators = []

    def fit(self, X, y):
        for i in range(self.n_estimators):
            
            np.random.seed(seed=self.random_state)
            
            s = sample(X, self.subspaces_dim)
            bootstrap_indices = s.bootstrap_sample(X)
            X_sampled, y_sampled = s.get_subsample(X, y, s.idx_subspace, bootstrap_indices)
            
            clf = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            clf.fit(X_sampled, y_sampled)
            
            self._estimators_params.append(s.idx_subspace)
            self._estimators.append(clf)

    def predict(self, X):
        answers = []
        for idx_subspace, estimator in zip(self._estimators_params, self._estimators):
            predictions = estimator.predict(X[:, idx_subspace])
            answers.append(predictions)
        return np.ceil(np.array(answers).mean(axis=0))

In [119]:
from itertools import product
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook


def grid_search(random_state, *kwargs):
    
    res = []
    combinations = list(product(*kwargs))
    
    for combination in tqdm(combinations):
        
        max_depth, n_estimators, subspace_dim = combination
        
        clf = random_forest(n_estimators, max_depth, subspace_dim, random_state=random_state)
        clf.fit(X_train, y_train)
        
        accuracy = accuracy_score(clf.predict(X_test), y_test)
        res.append((accuracy, max_depth, n_estimators, subspace_dim))
    
    return res

In [127]:
max_depth_list = range(1, 100, 1)
n_estimators_list = range(1, 100, 1)
subspace_dim_list = [1, 2, 3, 4]

In [128]:
res = grid_search(2, max_depth_list, n_estimators_list, subspace_dim_list)
sorted(np.array(res), key=lambda x: -x[0])

100%|████████████████████████████████████| 39204/39204 [06:06<00:00, 106.82it/s]


[array([0.95238095, 2.        , 1.        , 1.        ]),
 array([0.95238095, 2.        , 2.        , 1.        ]),
 array([0.95238095, 2.        , 3.        , 1.        ]),
 array([0.95238095, 2.        , 4.        , 1.        ]),
 array([0.95238095, 2.        , 5.        , 1.        ]),
 array([0.95238095, 2.        , 6.        , 1.        ]),
 array([0.95238095, 2.        , 7.        , 1.        ]),
 array([0.95238095, 2.        , 8.        , 1.        ]),
 array([0.95238095, 2.        , 9.        , 1.        ]),
 array([ 0.95238095,  2.        , 10.        ,  1.        ]),
 array([ 0.95238095,  2.        , 11.        ,  1.        ]),
 array([ 0.95238095,  2.        , 12.        ,  1.        ]),
 array([ 0.95238095,  2.        , 13.        ,  1.        ]),
 array([ 0.95238095,  2.        , 14.        ,  1.        ]),
 array([ 0.95238095,  2.        , 15.        ,  1.        ]),
 array([ 0.95238095,  2.        , 16.        ,  1.        ]),
 array([ 0.95238095,  2.        , 17.       